In [2]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.3MB 10.3MB/s ta 0:00:01
  Found existing installation: pip 18.0
    Uninstalling pip-18.0:
      Successfully uninstalled pip-18.0


In [3]:
namelist=['Abkhazia',
 'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua',
 'Argentina',
 'Armenia',
 'Artsakh',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'burkina faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'CookIslands',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech',
 'Democratic Republic of the Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'dominicana',
 'East Timor',
 'Ecuador',
 'egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'estonia',
 'Ethiopia',
 'Federated States of Micronesia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'greece',
 'Grenada',
 'Guatemala',
 'Guinea-Bissau',
 'Guinea',
 'Guyana',
 'Haiti',
 'Honduras',
 'hungary',
 'Iceland',
 'India',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Ivory Coast',
 'Jamaica',
 'Japan',
 'Jordan',
 'kazakstan',
 'Kenya',
 'Kilowatt',
 'Kiribati',
 'Kosovo',
 'ks',
 'Kyrgyz',
 'Laos',
 'Latvia',
 'Lebanon',
 'Lesotho',
 'Liberia',
 'Libya',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Madagascar',
 'Malawi',
 'Malaysia',
 'Maldives',
 'Mali',
 'Malta',
 'Marshall ',
 'Mauritania',
 'Mauritius',
 'Mexico',
 'mk',
 'Moldova',
 'Monaco',
 'Mongolia',
 'Montenegro',
 'Morocco',
 'Mozambique',
 'myanmar',
 'Namibia',
 'Nauru',
 'Nepal',
 'netherlands',
 'New Zealand',
 'Nicaragua',
 'Niger',
 'Nigeria',
 'Niue',
 'North korea',
 'Norway',
 'Oman',
 'Pakistan',
 'Palau',
 'Panama',
 'Paraguay',
 'Peru',
 'pg',
 'philippine',
 'Poland',
 'Portugal',
 'Qatar',
 'Republic of the Congo',
 'Romania',
 'Russia',
 'Rwanda',
 'Sahrawi_Arab_Democratic_Republic',
 'Saint Kitts and Nevis',
 'saint lucia',
 'Saint Vincent and the Grenadines',
 'Samoa',
 'San Marino',
 'Sao Tome and Principe',
 'Saudi Arabia',
 'Senegal',
 'Serbia',
 'Seychelles',
 'Sierra Leone',
 'singapore',
 'Slovakia',
 'slovenia',
 'Solomon Islands',
 'somalia',
 'Somaliland',
 'South Africa',
 'south korea',
 'South_Ossetia',
 'Spain',
 'Sri Lanka',
 'sudan',
 'Suriname',
 'Swaziland',
 'sweden',
 'Switzerland',
 'syrian',
 'Tajikistan',
 'Tanzania',
 'Thailand',
 'Togo',
 'Tonga',
 'Transnistria',
 'Trinidad',
 'Tunisia',
 'Turkey',
 'Turkish_Republic_of_Northern_Cyprus',
 'Turkmenistan',
 'Tuvalu',
 'Uganda',
 'Ukraine',
 'United Arab Emirates',
 'United Kingdom',
 'United States',
 'Uruguay',
 'Uzbekistan',
 'Vanuatu',
 'Vatican',
 'Venezuela',
 'Vietnam',
 'western sahara',
 'Yemen',
 'Zambia',
 'Zimbabwe']

In [4]:
namelist[0]

'Abkhazia'

In [ ]:
# -*- coding: utf-8 -*-

####################################################################################################################
# Download images from google with specified keywords for searching
# search query is created by "main_keyword + supplemented_keyword"
# if there are multiple keywords, each main_keyword will join with each supplemented_keyword
# mainly use urllib, and each search query will download at most 100 images due to page source code limited by google
# allow single process or multiple processes for downloading
####################################################################################################################
import os
import time
import re
import logging
import urllib.request
import urllib.error

from multiprocessing import Pool
from user_agent import generate_user_agent


log_file = 'download.log'
logging.basicConfig(level=logging.DEBUG, filename=log_file, filemode="a+", format="%(asctime)-15s %(levelname)-8s  %(message)s")


def download_page(url):
    """download raw content of the page
    
    Args:
        url (str): url of the page 
    
    Returns:
        raw content of the page
    """
    try:
        headers = {}
        headers['User-Agent'] = generate_user_agent()
        headers['Referer'] = 'https://www.google.com'
        req = urllib.request.Request(url, headers = headers)
        resp = urllib.request.urlopen(req)
        return str(resp.read())
    except Exception as e:
        print('error while downloading page {0}'.format(url))
        logging.error('error while downloading page {0}'.format(url))
        return None


def parse_page(url):
    """parge the page and get all the links of images, max number is 100 due to limit by google
    
    Args:
        url (str): url of the page
    
    Returns:
        A set containing the urls of images
    """
    page_content = download_page(url)
    if page_content:
        link_list = re.findall('"ou":"(.*?)"', page_content)
        if len(link_list) == 0:
            print('get 0 links from page {0}'.format(url))
            logging.info('get 0 links from page {0}'.format(url))
            return set()
        else:
            return set(link_list)
    else:
        return set()


def download_images(main_keyword, supplemented_keywords, download_dir):
    """download images with one main keyword and multiple supplemented keywords
    
    Args:
        main_keyword (str): main keyword
        supplemented_keywords (list[str]): list of supplemented keywords
    
    Returns:
        None
    """  
    image_links = set()
    print('Process {0} Main keyword: {1}'.format(os.getpid(), main_keyword))

    # create a directory for a main keyword
    img_dir =  download_dir + main_keyword + '/'
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)

    for j in range(len(supplemented_keywords)):
        print('Process {0} supplemented keyword: {1}'.format(os.getpid(), supplemented_keywords[j]))
        search_query = (main_keyword + ' ' + supplemented_keywords[j]).replace(' ','%20')
        # url = 'https://www.google.com/search?q=' + search_query + '&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg'
        url = 'https://www.google.com/search?q=' + search_query + '&source=lnms&tbm=isch'
        image_links = image_links.union(parse_page(url))
        print('Process {0} get {1} links so far'.format(os.getpid(), len(image_links)))
        time.sleep(2)
    print ("Process {0} get totally {1} links".format(os.getpid(), len(image_links)))

    print ("Start downloading...")
    count = 1
    for link in image_links:
        try:
            req = urllib.request.Request(link, headers = {"User-Agent": generate_user_agent()})
            response = urllib.request.urlopen(req)
            data = response.read()
            file_path = img_dir + '{0}.jpg'.format(count)
            with open(file_path,'wb') as wf:
                wf.write(data)
            print('Process {0} fininsh image {1}/{2}.jpg'.format(os.getpid(), main_keyword, count))
            count += 1
        except urllib.error.URLError as e:
            logging.error('URLError while downloading image {0}\nreason:{1}'.format(link, e.reason))
            continue
        except urllib.error.HTTPError as e:
            logging.error('HTTPError while downloading image {0}\nhttp code {1}, reason:{2}'.format(link, e.code, e.reason))
            continue
        except Exception as e:
            logging.error('Unexpeted error while downloading image {0}\nerror type:{1}, args:{2}'.format(link, type(e), e.args))
            continue

    print("Finish downloading, total {0} errors".format(len(image_links) - count))
    


if __name__ == '__main__':
    main_keywords = namelist

    supplemented_keywords = ['flag']

    download_dir = './data/'

    # download with single process
    # for i in range(len(main_keywords)):
    #     download_images(main_keywords[i], supplemented_keywords, download_dir)


    # download with multiple process
    p = Pool() # number of process is the number of cores of your CPU
    for i in range(len(main_keywords)):
        p.apply_async(download_images, args=(main_keywords[i], supplemented_keywords, download_dir))
    p.close()
    p.join()
    print('All fininshed')


Process 2743 Main keyword: Abkhazia
Process 2743 supplemented keyword: flag
Process 2743 get 100 links so far
Process 2743 get totally 100 links
Start downloading...
Process 2743 fininsh image Abkhazia/1.jpg
Process 2743 fininsh image Abkhazia/2.jpg
Process 2743 fininsh image Abkhazia/3.jpg
Process 2743 fininsh image Abkhazia/4.jpg
Process 2743 fininsh image Abkhazia/5.jpg
Process 2743 fininsh image Abkhazia/6.jpg
Process 2743 fininsh image Abkhazia/7.jpg
Process 2743 fininsh image Abkhazia/8.jpg
Process 2743 fininsh image Abkhazia/9.jpg
Process 2743 fininsh image Abkhazia/10.jpg
Process 2743 fininsh image Abkhazia/11.jpg
Process 2743 fininsh image Abkhazia/12.jpg
Process 2743 fininsh image Abkhazia/13.jpg
Process 2743 fininsh image Abkhazia/14.jpg
Process 2743 fininsh image Abkhazia/15.jpg
Process 2743 fininsh image Abkhazia/16.jpg
Process 2743 fininsh image Abkhazia/17.jpg
Process 2743 fininsh image Abkhazia/18.jpg
Process 2743 fininsh image Abkhazia/19.jpg
Process 2743 fininsh image

Process 2743 fininsh image Afghanistan/78.jpg
Process 2743 fininsh image Afghanistan/79.jpg
Process 2743 fininsh image Afghanistan/80.jpg
Process 2743 fininsh image Afghanistan/81.jpg
Process 2743 fininsh image Afghanistan/82.jpg
Process 2743 fininsh image Afghanistan/83.jpg
Process 2743 fininsh image Afghanistan/84.jpg
Process 2743 fininsh image Afghanistan/85.jpg
Process 2743 fininsh image Afghanistan/86.jpg
Process 2743 fininsh image Afghanistan/87.jpg
Process 2743 fininsh image Afghanistan/88.jpg
Process 2743 fininsh image Afghanistan/89.jpg
Process 2743 fininsh image Afghanistan/90.jpg
Process 2743 fininsh image Afghanistan/91.jpg
Process 2743 fininsh image Afghanistan/92.jpg
Process 2743 fininsh image Afghanistan/93.jpg
Process 2743 fininsh image Afghanistan/94.jpg
Process 2743 fininsh image Afghanistan/95.jpg
Process 2743 fininsh image Afghanistan/96.jpg
Process 2743 fininsh image Afghanistan/97.jpg
Process 2743 fininsh image Afghanistan/98.jpg
Process 2743 fininsh image Afghani

Process 2743 fininsh image Algeria/62.jpg
Process 2743 fininsh image Algeria/63.jpg
Process 2743 fininsh image Algeria/64.jpg
Process 2743 fininsh image Algeria/65.jpg
Process 2743 fininsh image Algeria/66.jpg
Process 2743 fininsh image Algeria/67.jpg
Process 2743 fininsh image Algeria/68.jpg
Process 2743 fininsh image Algeria/69.jpg
Process 2743 fininsh image Algeria/70.jpg
Process 2743 fininsh image Algeria/71.jpg
Process 2743 fininsh image Algeria/72.jpg
Process 2743 fininsh image Algeria/73.jpg
Process 2743 fininsh image Algeria/74.jpg
Process 2743 fininsh image Algeria/75.jpg
Process 2743 fininsh image Algeria/76.jpg
Process 2743 fininsh image Algeria/77.jpg
Process 2743 fininsh image Algeria/78.jpg
Process 2743 fininsh image Algeria/79.jpg
Process 2743 fininsh image Algeria/80.jpg
Process 2743 fininsh image Algeria/81.jpg
Process 2743 fininsh image Algeria/82.jpg
Process 2743 fininsh image Algeria/83.jpg
Process 2743 fininsh image Algeria/84.jpg
Process 2743 fininsh image Algeria

Process 2743 fininsh image Angola/54.jpg
Process 2743 fininsh image Angola/55.jpg
Process 2743 fininsh image Angola/56.jpg
Process 2743 fininsh image Angola/57.jpg
Process 2743 fininsh image Angola/58.jpg
Process 2743 fininsh image Angola/59.jpg
Process 2743 fininsh image Angola/60.jpg
Process 2743 fininsh image Angola/61.jpg
Process 2743 fininsh image Angola/62.jpg
Process 2743 fininsh image Angola/63.jpg
Process 2743 fininsh image Angola/64.jpg
Process 2743 fininsh image Angola/65.jpg
Process 2743 fininsh image Angola/66.jpg
Process 2743 fininsh image Angola/67.jpg
Process 2743 fininsh image Angola/68.jpg
Process 2743 fininsh image Angola/69.jpg
Process 2743 fininsh image Angola/70.jpg
Process 2743 fininsh image Angola/71.jpg
Process 2743 fininsh image Angola/72.jpg
Process 2743 fininsh image Angola/73.jpg
Process 2743 fininsh image Angola/74.jpg
Process 2743 fininsh image Angola/75.jpg
Process 2743 fininsh image Angola/76.jpg
Process 2743 fininsh image Angola/77.jpg
Process 2743 fin

Process 2743 fininsh image Argentina/43.jpg
Process 2743 fininsh image Argentina/44.jpg
Process 2743 fininsh image Argentina/45.jpg
Process 2743 fininsh image Argentina/46.jpg
Process 2743 fininsh image Argentina/47.jpg
Process 2743 fininsh image Argentina/48.jpg
Process 2743 fininsh image Argentina/49.jpg
Process 2743 fininsh image Argentina/50.jpg
Process 2743 fininsh image Argentina/51.jpg
Process 2743 fininsh image Argentina/52.jpg
Process 2743 fininsh image Argentina/53.jpg
Process 2743 fininsh image Argentina/54.jpg
Process 2743 fininsh image Argentina/55.jpg
Process 2743 fininsh image Argentina/56.jpg
Process 2743 fininsh image Argentina/57.jpg
Process 2743 fininsh image Argentina/58.jpg
Process 2743 fininsh image Argentina/59.jpg
Process 2743 fininsh image Argentina/60.jpg
Process 2743 fininsh image Argentina/61.jpg
Process 2743 fininsh image Argentina/62.jpg
Process 2743 fininsh image Argentina/63.jpg
Process 2743 fininsh image Argentina/64.jpg
Process 2743 fininsh image Argen

...

Process 2743 fininsh image Morocco/33.jpg
Process 2743 fininsh image Morocco/34.jpg
Process 2743 fininsh image Morocco/35.jpg
Process 2743 fininsh image Morocco/36.jpg
Process 2743 fininsh image Morocco/37.jpg
Process 2743 fininsh image Morocco/38.jpg
Process 2743 fininsh image Morocco/39.jpg
Process 2743 fininsh image Morocco/40.jpg
Process 2743 fininsh image Morocco/41.jpg
Process 2743 fininsh image Morocco/42.jpg
Process 2743 fininsh image Morocco/43.jpg
Process 2743 fininsh image Morocco/44.jpg
Process 2743 fininsh image Morocco/45.jpg
Process 2743 fininsh image Morocco/46.jpg
Process 2743 fininsh image Morocco/47.jpg
Process 2743 fininsh image Morocco/48.jpg
Process 2743 fininsh image Morocco/49.jpg
Process 2743 fininsh image Morocco/50.jpg
Process 2743 fininsh image Morocco/51.jpg
Process 2743 fininsh image Morocco/52.jpg
Process 2743 fininsh image Morocco/53.jpg
Process 2743 fininsh image Morocco/54.jpg
Process 2743 fininsh image Morocco/55.jpg
Process 2743 fininsh image Morocco

Process 2743 fininsh image myanmar/17.jpg
Process 2743 fininsh image myanmar/18.jpg
Process 2743 fininsh image myanmar/19.jpg
Process 2743 fininsh image myanmar/20.jpg
Process 2743 fininsh image myanmar/21.jpg
Process 2743 fininsh image myanmar/22.jpg
Process 2743 fininsh image myanmar/23.jpg
Process 2743 fininsh image myanmar/24.jpg
Process 2743 fininsh image myanmar/25.jpg
Process 2743 fininsh image myanmar/26.jpg
Process 2743 fininsh image myanmar/27.jpg
Process 2743 fininsh image myanmar/28.jpg
Process 2743 fininsh image myanmar/29.jpg
Process 2743 fininsh image myanmar/30.jpg
Process 2743 fininsh image myanmar/31.jpg
Process 2743 fininsh image myanmar/32.jpg
Process 2743 fininsh image myanmar/33.jpg
Process 2743 fininsh image myanmar/34.jpg
Process 2743 fininsh image myanmar/35.jpg
Process 2743 fininsh image myanmar/36.jpg
Process 2743 fininsh image myanmar/37.jpg
Process 2743 fininsh image myanmar/38.jpg
Process 2743 fininsh image myanmar/39.jpg
Process 2743 fininsh image myanmar

Process 2743 fininsh image Nauru/3.jpg
Process 2743 fininsh image Nauru/4.jpg
Process 2743 fininsh image Nauru/5.jpg
Process 2743 fininsh image Nauru/6.jpg
Process 2743 fininsh image Nauru/7.jpg
Process 2743 fininsh image Nauru/8.jpg
Process 2743 fininsh image Nauru/9.jpg
Process 2743 fininsh image Nauru/10.jpg
Process 2743 fininsh image Nauru/11.jpg
Process 2743 fininsh image Nauru/12.jpg
Process 2743 fininsh image Nauru/13.jpg
Process 2743 fininsh image Nauru/14.jpg
Process 2743 fininsh image Nauru/15.jpg
Process 2743 fininsh image Nauru/16.jpg
Process 2743 fininsh image Nauru/17.jpg
Process 2743 fininsh image Nauru/18.jpg
Process 2743 fininsh image Nauru/19.jpg
Process 2743 fininsh image Nauru/20.jpg
Process 2743 fininsh image Nauru/21.jpg
Process 2743 fininsh image Nauru/22.jpg
Process 2743 fininsh image Nauru/23.jpg
Process 2743 fininsh image Nauru/24.jpg
Process 2743 fininsh image Nauru/25.jpg
Process 2743 fininsh image Nauru/26.jpg
Process 2743 fininsh image Nauru/27.jpg
Process

Process 2743 fininsh image netherlands/2.jpg
Process 2743 fininsh image netherlands/3.jpg
Process 2743 fininsh image netherlands/4.jpg
Process 2743 fininsh image netherlands/5.jpg
Process 2743 fininsh image netherlands/6.jpg
Process 2743 fininsh image netherlands/7.jpg
Process 2743 fininsh image netherlands/8.jpg
Process 2743 fininsh image netherlands/9.jpg
Process 2743 fininsh image netherlands/10.jpg
Process 2743 fininsh image netherlands/11.jpg
Process 2743 fininsh image netherlands/12.jpg
Process 2743 fininsh image netherlands/13.jpg
Process 2743 fininsh image netherlands/14.jpg
Process 2743 fininsh image netherlands/15.jpg
Process 2743 fininsh image netherlands/16.jpg
Process 2743 fininsh image netherlands/17.jpg
Process 2743 fininsh image netherlands/18.jpg
Process 2743 fininsh image netherlands/19.jpg
Process 2743 fininsh image netherlands/20.jpg
Process 2743 fininsh image netherlands/21.jpg
Process 2743 fininsh image netherlands/22.jpg
Process 2743 fininsh image netherlands/23.

Process 2743 fininsh image New Zealand/78.jpg
Process 2743 fininsh image New Zealand/79.jpg
Process 2743 fininsh image New Zealand/80.jpg
Process 2743 fininsh image New Zealand/81.jpg
Process 2743 fininsh image New Zealand/82.jpg
Process 2743 fininsh image New Zealand/83.jpg
Process 2743 fininsh image New Zealand/84.jpg
Process 2743 fininsh image New Zealand/85.jpg
Process 2743 fininsh image New Zealand/86.jpg
Process 2743 fininsh image New Zealand/87.jpg
Process 2743 fininsh image New Zealand/88.jpg
Process 2743 fininsh image New Zealand/89.jpg
Process 2743 fininsh image New Zealand/90.jpg
Process 2743 fininsh image New Zealand/91.jpg
Process 2743 fininsh image New Zealand/92.jpg
Process 2743 fininsh image New Zealand/93.jpg
Process 2743 fininsh image New Zealand/94.jpg
Process 2743 fininsh image New Zealand/95.jpg
Finish downloading, total 4 errors
Process 2743 Main keyword: Nicaragua
Process 2743 supplemented keyword: flag
Process 2743 get 100 links so far
Process 2743 get totally 1